<a href="https://colab.research.google.com/github/honomax/company_random_sampler/blob/main/%E4%BC%81%E6%A5%AD%E3%83%A9%E3%83%B3%E3%83%80%E3%83%A0%E6%8A%BD%E5%87%BA%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ライブラリのインポート

In [ ]:
import os
import pandas as pd

# データの読み込み

In [ ]:
base_dir = "/content/drive/MyDrive/company_random_sampler"  # 実行ファイルの場所
data_path = os.path.join(base_dir, "data", "EdinetcodeDlInfo.csv")

print(data_path)

#データの確認



In [ ]:
df = pd.read_csv(data_path, encoding='cp932', skiprows=1)

display(df.head())
print("DataFrameの列名:")
print(df.columns)
print(f"DataFrameの会社数: {df.shape[0]}件")

# 提出者業種の確認

In [ ]:
print(df['提出者業種'].unique())
print(f"Unique count: {df['提出者業種'].nunique()}")

#フィルタリングの設定

In [ ]:
# 複数条件によるフィルタリング
filtered_companies_df = df[
    (df['決算日'].astype(str) == '3月31日')&
    (df['上場区分'] == '上場') &
    (df['連結の有無'] == '有')
]

In [ ]:
#フィルタリング後の業種件数
print(filtered_companies_df['提出者業種'].unique())
print(f"Unique count: {filtered_companies_df['提出者業種'].nunique()}")

#企業ランダム抽出(重複なし)

In [ ]:
all_samples = []
random_seed_base = 42
selected_companies = set()  # 選ばれた企業を保存する集合

for i in range(5): #5回繰り返す

    # 前回までに選ばれた企業を除外
    df_available = filtered_companies_df[~filtered_companies_df['提出者名'].isin(selected_companies)]

    # 各業種から1件ずつサンプル抽出
    sample_i = df_available.groupby('提出者業種').sample(
        n=1,
        random_state=random_seed_base + i,
        replace=False
    )
    all_samples.append(sample_i)

    # 今回の抽出企業を追加
    selected_companies.update(sample_i['提出者名'].values)

final_samples = pd.concat(all_samples, ignore_index=True)

print(f"\n合計{len(final_samples)}件のデータを取得しました。(重複企業なし)")
display(final_samples.head(2))


#提出者名のユニーク件数が指定数未満の業種
industry_counts = filtered_companies_df.groupby('提出者業種')['提出者名'].nunique()
print(industry_counts[industry_counts < 5])

# ファイルの保存

In [ ]:
output_path = os.path.join(base_dir, "output.csv") # 出力先のパスとファイル名を指定
final_samples.to_csv(output_path, index=False, encoding='cp932')

print(f"'{output_path}' にファイルを出力しました。")